In [2]:
import prefect as pf
import prefect.blocks.system as pfbs


In [3]:
from polygon import RESTClient

In [4]:
polygon_key_block = await pfbs.Secret.load("polygon-api-key")
polygon_key = polygon_key_block.get()

In [5]:
minio_access_key_block = await pfbs.Secret.load("minio-access-key")
minio_access_key = minio_access_key_block.get()

minio_secret_key_block = await pfbs.Secret.load("minio-secret-key")
minio_secret_key = minio_secret_key_block.get()

In [6]:
plg = RESTClient(polygon_key)

In [7]:
date_from = "2023-01-01"
date_to = "2024-01-10"
ticker = "GE"


In [44]:
import polars as pl

In [21]:
aggs = plg.get_aggs(ticker, 1, "day", date_from, date_to)


In [64]:
from enum import Enum
import datetime
class BronzeConstants(Enum):
    TIMESTAMP_INSERTION_COLUMN = "ingestion_timestamp"


(
        pl.DataFrame(aggs)
        .with_columns(
            pl.lit(ticker).alias("ticker"),
            pl.lit(datetime.datetime.utcnow()).alias(BronzeConstants.TIMESTAMP_INSERTION_COLUMN.value),
            )
        )

open,high,low,close,volume,vwap,timestamp,transactions,otc,ticker,ingestion_timestamp
f64,f64,f64,f64,f64,f64,i64,i64,bool,str,datetime[μs]
84.05,85.09,83.54,84.98,6.405106e6,84.5049,1672722000000,76403,null,"""GE""",2024-02-11 21:02:20.876372
68.41,70.2,66.75,70.2,1.6784586e7,68.9452,1672808400000,150930,null,"""GE""",2024-02-11 21:02:20.876372
69.9,71.55,69.0,71.29,1.2770157e7,70.7197,1672894800000,104221,null,"""GE""",2024-02-11 21:02:20.876372
72.01,72.33,70.75,71.94,1.0389469e7,71.6373,1672981200000,80258,null,"""GE""",2024-02-11 21:02:20.876372
72.0,73.91,71.96,72.67,7.572883e6,72.9881,1673240400000,75631,null,"""GE""",2024-02-11 21:02:20.876372
72.2,75.37,72.2,75.27,9.255525e6,74.3053,1673326800000,75905,null,"""GE""",2024-02-11 21:02:20.876372
75.82,77.7,75.51,77.69,1.0159264e7,76.8927,1673413200000,90157,null,"""GE""",2024-02-11 21:02:20.876372
77.88,79.21,77.29,78.86,1.1498676e7,78.5699,1673499600000,74913,null,"""GE""",2024-02-11 21:02:20.876372
78.78,80.6,78.5,80.2,1.1048213e7,80.0552,1673586000000,83251,null,"""GE""",2024-02-11 21:02:20.876372


In [48]:
aggs_pl = pl.DataFrame(aggs).with_columns(ticker=pl.lit(ticker))

In [54]:
import os
os.environ["AWS_S3_ALLOW_UNSAFE_RENAME"] = "true"

In [52]:
polars_minio_credentials

{'access_key_id': 'Ate3PQ8y7T2JaGo6Xl0W',
 'secret_access_key': 'EDgaVIVrOYf55uFINY9xt9XLp6qoOmKx3L37yYQw',
 'endpoint_url': 'http://localhost:9000'}

In [61]:
aggs_pl.write_delta(f"s3://etl/{source}/aggs/", mode="append", storage_options={"region": "us-east-1", "allow_http": "true",**polars_minio_credentials})

In [62]:
pl.read_delta(f"s3://etl/{source}/aggs/", storage_options={"region": "us-east-1", "allow_http": "true",**polars_minio_credentials})

open,high,low,close,volume,vwap,timestamp,transactions,otc,ticker
f64,f64,f64,f64,f64,f64,i64,i64,bool,str
84.05,85.09,83.54,84.98,6.405106e6,84.5049,1672722000000,76403,null,"""GE"""
68.41,70.2,66.75,70.2,1.6784586e7,68.9452,1672808400000,150930,null,"""GE"""
69.9,71.55,69.0,71.29,1.2770157e7,70.7197,1672894800000,104221,null,"""GE"""
72.01,72.33,70.75,71.94,1.0389469e7,71.6373,1672981200000,80258,null,"""GE"""
72.0,73.91,71.96,72.67,7.572883e6,72.9881,1673240400000,75631,null,"""GE"""
72.2,75.37,72.2,75.27,9.255525e6,74.3053,1673326800000,75905,null,"""GE"""
75.82,77.7,75.51,77.69,1.0159264e7,76.8927,1673413200000,90157,null,"""GE"""
77.88,79.21,77.29,78.86,1.1498676e7,78.5699,1673499600000,74913,null,"""GE"""
78.78,80.6,78.5,80.2,1.1048213e7,80.0552,1673586000000,83251,null,"""GE"""


In [8]:
import json
import s3fs

In [36]:
from enum import Enum

class MedallionLayer(Enum):
    BRONZE = "bronze"
    SILVER = "silver"
    GOLD = "gold"

In [37]:

minio_fs = s3fs.S3FileSystem(
    key=minio_access_key,
    secret=minio_secret_key,
    client_kwargs={"endpoint_url": "http://localhost:9000"},
)

In [ ]:
polars_minio_credentials = {
    "access_key_id": minio_access_key,
    "secret_access_key": minio_secret_key,
    "endpoint_url": "http://localhost:9000",
}

In [38]:
source = "polygon"

minio_fs.ls(f"etl/{source}")

['etl/polygon/bronze', 'etl/polygon/raw', 'etl/polygon/silver']

In [41]:
 minio_fs.ls(f"{container_name}/{ticker}")

['etl/polygon/raw/GE/2023-01-01_2024-01-10.json']

In [43]:
pl.DataFrame(ticker_raw).with_columns(ticker=pl.lit(ticker), timestamp=pl.from_epoch("timestamp", time_unit="ms"))

open,high,low,close,volume,vwap,timestamp,transactions,otc,ticker
f64,f64,f64,f64,f64,f64,datetime[ms],i64,null,str
84.05,85.09,83.54,84.98,6.405106e6,84.5049,2023-01-03 05:00:00,76403,null,"""GE"""
68.41,70.2,66.75,70.2,1.6784586e7,68.9452,2023-01-04 05:00:00,150930,null,"""GE"""
69.9,71.55,69.0,71.29,1.2770157e7,70.7197,2023-01-05 05:00:00,104221,null,"""GE"""
72.01,72.33,70.75,71.94,1.0389469e7,71.6373,2023-01-06 05:00:00,80258,null,"""GE"""
72.0,73.91,71.96,72.67,7.572883e6,72.9881,2023-01-09 05:00:00,75631,null,"""GE"""
72.2,75.37,72.2,75.27,9.255525e6,74.3053,2023-01-10 05:00:00,75905,null,"""GE"""
75.82,77.7,75.51,77.69,1.0159264e7,76.8927,2023-01-11 05:00:00,90157,null,"""GE"""
77.88,79.21,77.29,78.86,1.1498676e7,78.5699,2023-01-12 05:00:00,74913,null,"""GE"""
78.78,80.6,78.5,80.2,1.1048213e7,80.0552,2023-01-13 05:00:00,83251,null,"""GE"""


In [59]:
with raw_fs.open(f"{container_name}/{ticker}/{date_from}_{date_to}.json", "w") as f:
    json.dump(list(agg.__dict__ for agg in aggs), f)

In [41]:
raw_fs.put(f"{container_name}/GE_1_day_2023-01-01_2024-01-10.json", json.dumps(list(agg.__dict__ for agg in aggs)))

ParamValidationError: Parameter validation failed:
Invalid bucket name "[{"open": 84.05, "high": 85.09, "low": 83.54, "close": 84.98, "volume": 6405106.0, "vwap": 84.5049, "timestamp": 1672722000000, "transactions": 76403, "otc": null}, {"open": 68.41, "high": 70.2, "low": 66.75, "close": 70.2, "volume": 16784586.0, "vwap": 68.9452, "timestamp": 1672808400000, "transactions": 150930, "otc": null}, {"open": 69.9, "high": 71.55, "low": 69, "close": 71.29, "volume": 12770157.0, "vwap": 70.7197, "timestamp": 1672894800000, "transactions": 104221, "otc": null}, {"open": 72.01, "high": 72.33, "low": 70.75, "close": 71.94, "volume": 10389469.0, "vwap": 71.6373, "timestamp": 1672981200000, "transactions": 80258, "otc": null}, {"open": 72, "high": 73.91, "low": 71.96, "close": 72.67, "volume": 7572883.0, "vwap": 72.9881, "timestamp": 1673240400000, "transactions": 75631, "otc": null}, {"open": 72.2, "high": 75.37, "low": 72.2, "close": 75.27, "volume": 9255525.0, "vwap": 74.3053, "timestamp": 1

In [32]:
json.dumps(list(agg.__dict__ for agg in aggs))

'[{"open": 84.05, "high": 85.09, "low": 83.54, "close": 84.98, "volume": 6405106.0, "vwap": 84.5049, "timestamp": 1672722000000, "transactions": 76403, "otc": null}, {"open": 68.41, "high": 70.2, "low": 66.75, "close": 70.2, "volume": 16784586.0, "vwap": 68.9452, "timestamp": 1672808400000, "transactions": 150930, "otc": null}, {"open": 69.9, "high": 71.55, "low": 69, "close": 71.29, "volume": 12770157.0, "vwap": 70.7197, "timestamp": 1672894800000, "transactions": 104221, "otc": null}, {"open": 72.01, "high": 72.33, "low": 70.75, "close": 71.94, "volume": 10389469.0, "vwap": 71.6373, "timestamp": 1672981200000, "transactions": 80258, "otc": null}, {"open": 72, "high": 73.91, "low": 71.96, "close": 72.67, "volume": 7572883.0, "vwap": 72.9881, "timestamp": 1673240400000, "transactions": 75631, "otc": null}, {"open": 72.2, "high": 75.37, "low": 72.2, "close": 75.27, "volume": 9255525.0, "vwap": 74.3053, "timestamp": 1673326800000, "transactions": 75905, "otc": null}, {"open": 75.82, "hig

In [ ]:
def aggs_to_json(aggs: list[dict])